#### Name: Tan Leong Jun Joseph (U2321339H), Tan Zhe Kai (U2322419A)

# Flight Delay Detection

Dataset from Kaggle: **"2019 Airline Delays w/ Weather and Airport Detail"** by *Jen Wadkins*

Source: https://www.kaggle.com/datasets/threnjen/2019-airline-delays-and-cancellations

# 1. Data Preparation & Cleaning <a id='1' />

## Table of Contents
1. [Introduction to Datasets](#1)
2. [Cleaning individual datasets](#2)
    - 2.1 [Flight Reporting Dataset](#2.1)
    - 2.2 [Aircraft Data](#2.2)
    - 2.3 [Airline Data](#2.3)
    - 2.4 [Passenger Data](#2.4)
    - 2.5 [Employee Data](#2.5)
    - 2.6 [Coordinates Data](#2.6)
    - 2.7 [Weather Data](#2.7)
    - 2.8 [Airport List Data](#2.8)
3. [Merge Weather & Airport List Data](#3)
4. [Merging of Datasets](#4)
5. [Data cleansing techniques used](#5)

# 1. Introduction to Datasets

Let's understand the data that is provided for us. We are given a total of **8** different types of datasets. The datasets are related such that each dataset contains a variable that can be found in another dataset, also known as a primary key.

**Main Flight Reporting Data**:
- `ONTIME_REPORTING` - monthly flight data

This dataset contains the flight records for every month for the period of January 2019 to March 2020. For this project, we will be using the flight reporting data from January to December 2019.

_Note: this dataset contains the target variable - `DEP_DEL15`, whether if the flight is delayed or not._

**Supporting Data**:
- `B43_AIRCRAFT_INVENTORY` - aircraft information
- `CARRIER_DECODE` - airline information
- `T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY` - passenger information
- `P10_EMPLOYEES` - employee information
- `AIRPORT_COORDINATES` - coordinates of airports
- `airport_weather` - information on the weather conditions per airport
- `airports_list` - information of airports

**Goal**:
To perform data preparation and cleansing techniques for individual data files and merge them to get the resulting cleaned data for exploratory data analysis and building of machine learning models.

In [8]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
sb.set()

## 2.1 Flight Reporting dataset - `ONTIME_REPORTING` <a id='2.1'/> 

Documentation of `ONTIME_REPORTING`:

>   **MONTH**: 			    Month  
>	**DAY_OF_MONTH**: 		Day of the month (1-31)  
>	**DAY_OF_WEEK**: 		Day of the week  
>	**OP_UNIQUE_CARRIER**: 	Carrier code, matches to OP_UNIQUE_CARRIER in other files  
>	**TAIL_NUM**: 		Unique tail number, matches to TAIL_NUM in other files  
>	**OP_CARRIER_FL_NUM**: 	Flight number  
>	**ORIGIN_AIRPORT_ID**: 	Airport ID, matches to ORIGIN_AIRPORT_ID in other files  
>	**ORIGIN**: 		Origin airport abbreviation  
>	**ORIGIN_CITY_NAME**: 	Origin city name  
>	**DEST_AIRPORT_ID**: 	Destination airport ID, matches Airport ID in other files  
>	**DEST**: 			Destination airport abbreviation  
>	**DEST_CITY_NAME**: 	Destination city name  
>	**CRS_DEP_TIME**: 		Planned departure time  
>	**DEP_TIME**: 		Actual departure time  
>	**DEP_DELAY_NEW**: 		Departure delay in minutes  
>	**DEP_DEL15**:		TARGET VARIABLE Binary if delayed over 15 min, 1 is yes  
>	**DEP_TIME_BLK**:		Departure time block  
>	**CRS_ARR_TIME**:		Planned arrival time  
>	**ARR_TIME**:		Actual arrival time  
>	**ARR_DELAY_NEW**:		Arrival delay in minutes  
>	**ARR_TIME_BLK**:		Arrival time block  
>	**CANCELLED**:		Flag if flight was cancelled  
>	**CANCELLATION_CODE**:	Cancellation Code  
>	**CRS_ELAPSED_TIME**:	Flight planned elapsed time  
>	**ACTUAL_ELAPSED_TIME**:	Flight actual elapsed time  
>	**DISTANCE**:		Flight Distance in miles  
>	**DISTANCE_GROUP**:		Flight distance group  
>	**CARRIER_DELAY**:		Flag for a carrier delay  
>	**WEATHER_DELAY**:		Flag for a weather delay  
>	**NAS_DELAY**:		Flag for a NAS delay  
>	**SECURITY_DELAY**:		Flag for a security delay  
>	**LATE_AIRCRAFT_DELAY**:	Flag for a late aircraft delay  

_Note: Column descriptions can be found in the file_

### Explore the first month of `ONTIME_REPORTING`

Before we clean all the `ONTIME_REPORTING` data files, let's understand what this dataset is all about.

In [9]:
# import one of the flight report dataset (Jan 2019)
data = pd.read_csv('raw_data/ONTIME_REPORTING_01.csv')

### Replacing blank cells with NaN using Regular Expression

In [10]:
# replace blanks with NaN
data = data.replace(r'^\s*$', np.nan, regex=True) 
data.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 32
0,1,6,7,9E,N8694A,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,37.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7,1,9E,N8970D,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,32.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
2,1,8,2,9E,N820AY,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,39.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,1,9,3,9E,N840AY,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,37.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10,4,9E,N8969A,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,41.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN


##### Shape and info

In [11]:
data.shape

(583985, 33)

In [12]:
# view the number of items and data type of each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583985 entries, 0 to 583984
Data columns (total 33 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   MONTH                583985 non-null  int64  
 1   DAY_OF_MONTH         583985 non-null  int64  
 2   DAY_OF_WEEK          583985 non-null  int64  
 3   OP_UNIQUE_CARRIER    583985 non-null  object 
 4   TAIL_NUM             581442 non-null  object 
 5   OP_CARRIER_FL_NUM    583985 non-null  int64  
 6   ORIGIN_AIRPORT_ID    583985 non-null  int64  
 7   ORIGIN               583985 non-null  object 
 8   ORIGIN_CITY_NAME     583985 non-null  object 
 9   DEST_AIRPORT_ID      583985 non-null  int64  
 10  DEST                 583985 non-null  object 
 11  DEST_CITY_NAME       583985 non-null  object 
 12  CRS_DEP_TIME         583985 non-null  int64  
 13  DEP_TIME             567633 non-null  float64
 14  DEP_DELAY_NEW        567630 non-null  float64
 15  DEP_DEL15        

### Removing NaN values
- As part of the data cleaning process, it is important that we identify the columns with NULL values and see what we can do with them.

In [13]:
# check for NaN values in each column
data.isnull().sum()

MONTH                       0
DAY_OF_MONTH                0
DAY_OF_WEEK                 0
OP_UNIQUE_CARRIER           0
TAIL_NUM                 2543
OP_CARRIER_FL_NUM           0
ORIGIN_AIRPORT_ID           0
ORIGIN                      0
ORIGIN_CITY_NAME            0
DEST_AIRPORT_ID             0
DEST                        0
DEST_CITY_NAME              0
CRS_DEP_TIME                0
DEP_TIME                16352
DEP_DELAY_NEW           16355
DEP_DEL15               16355
DEP_TIME_BLK                0
CRS_ARR_TIME                0
ARR_TIME                17061
ARR_DELAY_NEW           18022
ARR_TIME_BLK                0
CANCELLED                   0
CANCELLATION_CODE      567259
CRS_ELAPSED_TIME          134
ACTUAL_ELAPSED_TIME     18022
DISTANCE                    0
DISTANCE_GROUP              0
CARRIER_DELAY          478763
WEATHER_DELAY          478763
NAS_DELAY              478763
SECURITY_DELAY         478763
LATE_AIRCRAFT_DELAY    478763
Unnamed: 32            583985
dtype: int

For the dataset `ONTIME_REPORTING_XX`, the variables with NULL values are
1. `TAIL_NUM`
2. `DEP_TIME`
3. `DEP_DELAY_NEW`
4. `DEP_DEL15`
5. `ARR_TIME`
6. `ARR_DELAY_NEW`
7. `CANCELLATION_CODE`
8. `CRS_ELAPSED_TIME`
9. `CARRIER_DELAY`
10. `WEATHER_DELAY`
11. `NAS_DELAY`
12. `SECURITY_DELAY`
13. `LATE_AIRCRAFT_DELAY`
14. Unnamed: 32

**Observations**
- There is 1 unnamed column
- Flag delay variables (e.g. `CARRIER_DELAY`, `WEATHER_DELAY` etc) have same number of NaN values
- Cancellation code has many NaN values

### Dropping unnamed columns

In [14]:
# drop any unnamed columns
data.drop(data.columns[data.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)

### Explore flag delay variables

E.g. `CARRIER_DELAY`,`WEATHER_DELAY`,`NAS_DELAY`,`SECURITY_DELAY`,`LATE_AIRCRAFT_DELAY`

In [15]:
# display the relationship between the target variable 'DEP_DEL15' and the individual flag delay reasons
data.loc[data['DEP_DEL15']==1, ['DEP_DEL15', 'CARRIER_DELAY', 'WEATHER_DELAY', 
                                'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']]

,DEP_DEL15,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
6,1.0,18.0,0.0,0.0,0.0,6.0
9,1.0,132.0,0.0,0.0,0.0,0.0
14,1.0,100.0,0.0,0.0,0.0,1.0
29,1.0,0.0,0.0,0.0,0.0,16.0
32,1.0,0.0,0.0,0.0,0.0,117.0
...,...,...,...,...,...,...
583961,1.0,0.0,0.0,0.0,0.0,118.0
583966,1.0,0.0,26.0,0.0,0.0,0.0
583976,1.0,0.0,0.0,33.0,0.0,130.0
583979,1.0,0.0,0.0,73.0,0.0,0.0


The columns that indicate the type of delay is not useful as they are not binary coded like the 'DEP_DELAY15' column.
There is insufficient information from the documentation file to determine the meaning of the values. They could represent the period of delay in minutes or each value could represent a specific reason. Furthermore, some rows that are marked delayed have no values for the respective columns, meaning that there is no single reason delay filled with values.

Due to the inconsistency in the column data ('CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'), they will be dropped.

In [16]:
# drop the columns that already 
data.drop(columns = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 
                     'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'], axis = 1, inplace = True)

### Explore cancelled flights

In [17]:
# check the number of flights that were cancelled
data.loc[data['CANCELLED']==1, ['CANCELLED', 'CANCELLATION_CODE']]

,CANCELLED,CANCELLATION_CODE
130,1.0,B
270,1.0,B
338,1.0,B
836,1.0,B
974,1.0,B
...,...,...
583884,1.0,B
583939,1.0,B
583940,1.0,B
583943,1.0,B


In [18]:
# find out the unique cancellation codes
data['CANCELLATION_CODE'].unique()

array([nan, 'B', 'C', 'A', 'D'], dtype=object)

In [19]:
# find out the number of each unique cancellation code
data['CANCELLATION_CODE'].value_counts()

CANCELLATION_CODE
B    11900
C     2908
A     1917
D        1
Name: count, dtype: int64

There were 16,726 flights that were cancelled. Each cancelled flight is marked with a cancellation code either A, B, C or D.

The problem is about flights delay, therefore the data representing flight cancellations is irrelevant.

In [20]:
# drop rows that represent cancelled flights
data.drop(data.loc[data['CANCELLED']==1].index, axis=0, inplace=True)

In [21]:
data.isnull().sum()

MONTH                       0
DAY_OF_MONTH                0
DAY_OF_WEEK                 0
OP_UNIQUE_CARRIER           0
TAIL_NUM                    0
OP_CARRIER_FL_NUM           0
ORIGIN_AIRPORT_ID           0
ORIGIN                      0
ORIGIN_CITY_NAME            0
DEST_AIRPORT_ID             0
DEST                        0
DEST_CITY_NAME              0
CRS_DEP_TIME                0
DEP_TIME                    0
DEP_DELAY_NEW               0
DEP_DEL15                   0
DEP_TIME_BLK                0
CRS_ARR_TIME                0
ARR_TIME                  335
ARR_DELAY_NEW            1296
ARR_TIME_BLK                0
CANCELLED                   0
CANCELLATION_CODE      567259
CRS_ELAPSED_TIME            0
ACTUAL_ELAPSED_TIME      1296
DISTANCE                    0
DISTANCE_GROUP              0
dtype: int64

### Dropping unnecessary columns

In [22]:
# drop the unused columns
data.drop(columns = ['CANCELLED','CANCELLATION_CODE','CRS_ELAPSED_TIME','ACTUAL_ELAPSED_TIME',
                     'ARR_TIME','ARR_DELAY_NEW','ARR_TIME_BLK','CRS_ARR_TIME',
                     'CRS_DEP_TIME','ORIGIN','DEST'], axis=1, inplace=True)

In [23]:
data

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,DISTANCE,DISTANCE_GROUP
0,1,6,7,9E,N8694A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1643.0,0.0,0.0,1600-1659,83.0,1
1,1,7,1,9E,N8970D,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1
2,1,8,2,9E,N820AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1
3,1,9,3,9E,N840AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1
4,1,10,4,9E,N8969A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583980,1,30,3,UA,N819UA,2024,14683,"San Antonio, TX",12266,"Houston, TX",1420.0,0.0,0.0,1400-1459,191.0,1
583981,1,30,3,UA,N37462,2022,14843,"San Juan, PR",12264,"Washington, DC",1407.0,0.0,0.0,1400-1459,1571.0,7
583982,1,30,3,UA,N37462,2021,12264,"Washington, DC",14843,"San Juan, PR",819.0,0.0,0.0,0800-0859,1571.0,7
583983,1,30,3,UA,N26967,2020,12266,"Houston, TX",14771,"San Francisco, CA",1858.0,43.0,1.0,1800-1859,1635.0,7


### Feature Engineering 

- Using selected columns, we can transform them to create new columns for further data analysis.
- We created new column variables `SEGMENT_NUMBER` and `CONCURRENT_FLIGHTS` using the groupby function.

In [24]:
# create new variable 'SEGMENT_NUMBER' 
# - the segment that this tail number is on for the day
data["SEGMENT_NUMBER"] = data.groupby(["TAIL_NUM", 'DAY_OF_MONTH'])["DEP_TIME"].rank("dense", ascending=True)

# create new variable 'CONCURRENT_FLIGHTS' 
# - number of concurrent flights leaving from the same airport and departure time block
data['CONCURRENT_FLIGHTS'] = data.groupby(['ORIGIN_AIRPORT_ID','DAY_OF_MONTH', 'DEP_TIME_BLK'])['OP_UNIQUE_CARRIER'].transform("count")

In [25]:
data

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,DISTANCE,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS
0,1,6,7,9E,N8694A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1643.0,0.0,0.0,1600-1659,83.0,1,3.0,76
1,1,7,1,9E,N8970D,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1,6.0,71
2,1,8,2,9E,N820AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1,5.0,69
3,1,9,3,9E,N840AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1,6.0,70
4,1,10,4,9E,N8969A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1,6.0,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583980,1,30,3,UA,N819UA,2024,14683,"San Antonio, TX",12266,"Houston, TX",1420.0,0.0,0.0,1400-1459,191.0,1,3.0,7
583981,1,30,3,UA,N37462,2022,14843,"San Juan, PR",12264,"Washington, DC",1407.0,0.0,0.0,1400-1459,1571.0,7,2.0,9
583982,1,30,3,UA,N37462,2021,12264,"Washington, DC",14843,"San Juan, PR",819.0,0.0,0.0,0800-0859,1571.0,7,1.0,36
583983,1,30,3,UA,N26967,2020,12266,"Houston, TX",14771,"San Francisco, CA",1858.0,43.0,1.0,1800-1859,1635.0,7,2.0,58


In [26]:
# Check for any null values in new columns
data.isnull().sum()

MONTH                 0
DAY_OF_MONTH          0
DAY_OF_WEEK           0
OP_UNIQUE_CARRIER     0
TAIL_NUM              0
OP_CARRIER_FL_NUM     0
ORIGIN_AIRPORT_ID     0
ORIGIN_CITY_NAME      0
DEST_AIRPORT_ID       0
DEST_CITY_NAME        0
DEP_TIME              0
DEP_DELAY_NEW         0
DEP_DEL15             0
DEP_TIME_BLK          0
DISTANCE              0
DISTANCE_GROUP        0
SEGMENT_NUMBER        0
CONCURRENT_FLIGHTS    0
dtype: int64

In [27]:
data

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,DISTANCE,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS
0,1,6,7,9E,N8694A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1643.0,0.0,0.0,1600-1659,83.0,1,3.0,76
1,1,7,1,9E,N8970D,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1,6.0,71
2,1,8,2,9E,N820AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1,5.0,69
3,1,9,3,9E,N840AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1,6.0,70
4,1,10,4,9E,N8969A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",1640.0,0.0,0.0,1600-1659,83.0,1,6.0,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583980,1,30,3,UA,N819UA,2024,14683,"San Antonio, TX",12266,"Houston, TX",1420.0,0.0,0.0,1400-1459,191.0,1,3.0,7
583981,1,30,3,UA,N37462,2022,14843,"San Juan, PR",12264,"Washington, DC",1407.0,0.0,0.0,1400-1459,1571.0,7,2.0,9
583982,1,30,3,UA,N37462,2021,12264,"Washington, DC",14843,"San Juan, PR",819.0,0.0,0.0,0800-0859,1571.0,7,1.0,36
583983,1,30,3,UA,N26967,2020,12266,"Houston, TX",14771,"San Francisco, CA",1858.0,43.0,1.0,1800-1859,1635.0,7,2.0,58


## 2.2  Aircraft Data - `B43_AIRCRAFT_INVENTORY` <a id='2.2' /> 

The aircraft dataset contains information about the aircraft such as the year of manufacture, number of seats and the given tail number.

In [21]:
aircraft = pd.read_csv('raw_data/B43_AIRCRAFT_INVENTORY.csv', encoding='latin1')
aircraft

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0
...,...,...,...
7378,2019,N14011,337.0
7379,2019,N16008,337.0
7380,2019,N16009,337.0
7381,2019,N2250U,276.0


### Checking for null values

In [22]:
aircraft.isnull().sum()

MANUFACTURE_YEAR    0
TAIL_NUM            0
NUMBER_OF_SEATS     7
dtype: int64

### Checking for duplicated `TAIL_NUM`

In [23]:
aircraft[aircraft['TAIL_NUM'].duplicated(keep=False)].sort_values(by='TAIL_NUM')

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
1544,1999,N1049A,269.0
1547,1999,N1049A,0.0
1210,1997,N1439A,269.0
1213,1997,N1439A,0.0
4406,2008,N235JQ,76.0
4519,2008,N235JQ,76.0
770,1993,N399CM,0.0
845,1993,N399CM,0.0
2903,2002,N416MC,0.0
2694,2002,N416MC,0.0


### Dropping duplicate `TAIL_NUM`

In [24]:
aircraft.drop_duplicates(subset='TAIL_NUM', inplace=True)
aircraft

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0
...,...,...,...
7378,2019,N14011,337.0
7379,2019,N16008,337.0
7380,2019,N16009,337.0
7381,2019,N2250U,276.0


### Merging with the main dataset

In [25]:
data = pd.merge(data, aircraft, how='left', on='TAIL_NUM')

In [26]:
data.isnull().sum()

MONTH                     0
DAY_OF_MONTH              0
DAY_OF_WEEK               0
OP_UNIQUE_CARRIER         0
TAIL_NUM                  0
OP_CARRIER_FL_NUM         0
ORIGIN_AIRPORT_ID         0
ORIGIN_CITY_NAME          0
DEST_AIRPORT_ID           0
DEST_CITY_NAME            0
DEP_TIME                  0
DEP_DELAY_NEW             0
DEP_DEL15                 0
DEP_TIME_BLK              0
DISTANCE                  0
DISTANCE_GROUP            0
SEGMENT_NUMBER            0
CONCURRENT_FLIGHTS        0
MANUFACTURE_YEAR      32139
NUMBER_OF_SEATS       32139
dtype: int64

## 2.3 Airline Data - `CARRIER_DECODE` <a id='2.3'/>

This dataset contains the name of airlines and their unique codes.

In [27]:
airline = pd.read_csv('raw_data/CARRIER_DECODE.csv')
airline

,AIRLINE_ID,OP_UNIQUE_CARRIER,CARRIER_NAME
0,21754,2PQ,21 Air LLC
1,21754,2PQ,21 Air LLC
2,21754,2PQ,21 Air LLC
3,20342,Q5,40-Mile Air
4,20342,WRB,40-Mile Air
...,...,...,...
2705,20379,ZKQ,Zantop International
2706,19771,ZAQ,Zas Airline Of Egypt
2707,21118,37,Zeal 320
2708,22069,ZG,ZIPAIR Tokyo Inc.


### Check for NULL values

In [28]:
airline.isnull().sum()

AIRLINE_ID           0
OP_UNIQUE_CARRIER    4
CARRIER_NAME         0
dtype: int64

### Check for duplicated `AIRLINE_ID`

In [29]:
airline[airline['AIRLINE_ID'].duplicated(keep=False)].sort_values(by='AIRLINE_ID')

,AIRLINE_ID,OP_UNIQUE_CARRIER,CARRIER_NAME
1748,19386,NW,Northwest Airlines Inc.
1749,19386,NW,Northwest Airlines Inc.
1750,19386,NW,Northwest Airlines Inc.
1751,19386,NW,Northwest Airlines Inc.
191,19389,QH,Air Florida Inc.
...,...,...,...
2258,22035,LDQ,Sparfell Luftfahrt GmbH
23,22036,X7,ACE Belgium Freighters S.A.
755,22036,X7,Challenge Airlines (BE) S.A.
1419,22075,3KQ,"Junipogo, LLC"


### Dropping duplicate `AIRLINE_ID`

In [30]:
airline.drop_duplicates(subset='AIRLINE_ID', inplace=True)
airline

,AIRLINE_ID,OP_UNIQUE_CARRIER,CARRIER_NAME
0,21754,2PQ,21 Air LLC
3,20342,Q5,40-Mile Air
6,19627,CIQ,A/S Conair
7,19072,AAE,AAA Airlines
8,19079,ACI,AAA-Action Air Carrier Inc.
...,...,...,...
2702,20379,ZKQ,Zantop International
2706,19771,ZAQ,Zas Airline Of Egypt
2707,21118,37,Zeal 320
2708,22069,ZG,ZIPAIR Tokyo Inc.


### Check for duplicate `OP_UNIQUE_CARRIER` code

In [31]:
airline[airline['OP_UNIQUE_CARRIER'].duplicated(keep=False)].sort_values(by='OP_UNIQUE_CARRIER')

,AIRLINE_ID,OP_UNIQUE_CARRIER,CARRIER_NAME
606,21599,4B,Boutique Air
1768,20318,4B,Olson Air Service
1494,20350,4M,Lan Dominica
1492,21084,4M,LAN Argentina
1987,20165,5G,Queen Air
...,...,...,...
578,20322,Z3,"Bidzy Ta Hot Aana, Inc. d/b/a Tanana Air Service"
1521,20230,ZE,Lineas Aereas Azteca
944,21781,ZE,Eastar Jet Co Ltd.
196,20293,ZX,Air Georgian


### Dropping duplicate `OP_UNIQUE_CARRIER` codes

In [32]:
airline.drop_duplicates(subset='OP_UNIQUE_CARRIER', inplace=True)
airline

,AIRLINE_ID,OP_UNIQUE_CARRIER,CARRIER_NAME
0,21754,2PQ,21 Air LLC
3,20342,Q5,40-Mile Air
6,19627,CIQ,A/S Conair
7,19072,AAE,AAA Airlines
8,19079,ACI,AAA-Action Air Carrier Inc.
...,...,...,...
2702,20379,ZKQ,Zantop International
2706,19771,ZAQ,Zas Airline Of Egypt
2707,21118,37,Zeal 320
2708,22069,ZG,ZIPAIR Tokyo Inc.


### Merging with the main dataset

In [33]:
data = pd.merge(data, airline, how='left', on='OP_UNIQUE_CARRIER')
data

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,...,DEP_DEL15,DEP_TIME_BLK,DISTANCE,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,MANUFACTURE_YEAR,NUMBER_OF_SEATS,AIRLINE_ID,CARRIER_NAME
0,1,6,7,9E,N8694A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,0.0,1600-1659,83.0,1,3.0,76,2002.0,50.0,20363,Endeavor Air Inc.
1,1,7,1,9E,N8970D,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,0.0,1600-1659,83.0,1,6.0,71,2004.0,50.0,20363,Endeavor Air Inc.
2,1,8,2,9E,N820AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,0.0,1600-1659,83.0,1,5.0,69,2005.0,50.0,20363,Endeavor Air Inc.
3,1,9,3,9E,N840AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,0.0,1600-1659,83.0,1,6.0,70,2005.0,50.0,20363,Endeavor Air Inc.
4,1,10,4,9E,N8969A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,0.0,1600-1659,83.0,1,6.0,70,2004.0,50.0,20363,Endeavor Air Inc.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567254,1,30,3,UA,N819UA,2024,14683,"San Antonio, TX",12266,"Houston, TX",...,0.0,1400-1459,191.0,1,3.0,7,1998.0,120.0,19977,United Air Lines Inc.
567255,1,30,3,UA,N37462,2022,14843,"San Juan, PR",12264,"Washington, DC",...,0.0,1400-1459,1571.0,7,2.0,9,2012.0,173.0,19977,United Air Lines Inc.
567256,1,30,3,UA,N37462,2021,12264,"Washington, DC",14843,"San Juan, PR",...,0.0,0800-0859,1571.0,7,1.0,36,2012.0,173.0,19977,United Air Lines Inc.
567257,1,30,3,UA,N26967,2020,12266,"Houston, TX",14771,"San Francisco, CA",...,1.0,1800-1859,1635.0,7,2.0,58,2016.0,219.0,19977,United Air Lines Inc.


## 2.4 Passenger Data - `T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY` <a id='2.4'/>

This dataset contains all passenger information corresponding to the different airlines for the particular year.

In [34]:
passenger = pd.read_csv('raw_data/T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY_2019.csv')
passenger

,OP_UNIQUE_CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,SERVICE_CLASS,REV_ACRFT_DEP_PERF_510,REV_PAX_ENP_110
0,04Q,Tradewind Aviation,15024,K,10.0,39.0
1,04Q,Tradewind Aviation,14843,K,677.0,3649.0
2,04Q,Tradewind Aviation,10257,V,4.0,6.0
3,04Q,Tradewind Aviation,15323,V,1.0,3.0
4,04Q,Tradewind Aviation,10158,V,1.0,2.0
...,...,...,...,...,...,...
27247,ZW,Air Wisconsin Airlines Corp,11637,K,122.0,4535.0
27248,ZW,Air Wisconsin Airlines Corp,11721,K,143.0,5800.0
27249,ZW,Air Wisconsin Airlines Corp,10469,K,248.0,8901.0
27250,ZW,Air Wisconsin Airlines Corp,12884,K,187.0,7923.0


### Check for NULL values

In [35]:
passenger.isnull().sum()

OP_UNIQUE_CARRIER            0
CARRIER_NAME                 0
ORIGIN_AIRPORT_ID            0
SERVICE_CLASS                0
REV_ACRFT_DEP_PERF_510       1
REV_PAX_ENP_110           1665
dtype: int64

### Feature Engineering

- Created two new variables to determine the monthly passengers by airport/airline.
- `monthly_airport_passengers`
- `monthly_airline_passengers`

In [36]:
monthly_airport_passengers = pd.DataFrame(passenger.groupby(['ORIGIN_AIRPORT_ID'])['REV_PAX_ENP_110'].sum())
monthly_airport_passengers

,REV_PAX_ENP_110
ORIGIN_AIRPORT_ID,
10005,24.0
10006,3.0
10009,16.0
10010,14.0
10011,18201.0
...,...
16871,1.0
16872,0.0
16873,351.0


In [37]:
monthly_airline_passengers = pd.DataFrame(passenger.groupby(['OP_UNIQUE_CARRIER'])['REV_PAX_ENP_110'].sum())
monthly_airline_passengers

,REV_PAX_ENP_110
OP_UNIQUE_CARRIER,
04Q,33777.0
09Q,174364.0
0WQ,706.0
1AQ,12332.0
1BQ,17810.0
...,...
YV,14302678.0
YX,18356886.0
Z3,5724.0


### Merge with main data file

In [38]:
data = pd.merge(data, monthly_airport_passengers, how='left', on='ORIGIN_AIRPORT_ID')
data

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,...,DEP_TIME_BLK,DISTANCE,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,MANUFACTURE_YEAR,NUMBER_OF_SEATS,AIRLINE_ID,CARRIER_NAME,REV_PAX_ENP_110
0,1,6,7,9E,N8694A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,1600-1659,83.0,1,3.0,76,2002.0,50.0,20363,Endeavor Air Inc.,52387940.0
1,1,7,1,9E,N8970D,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,1600-1659,83.0,1,6.0,71,2004.0,50.0,20363,Endeavor Air Inc.,52387940.0
2,1,8,2,9E,N820AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,1600-1659,83.0,1,5.0,69,2005.0,50.0,20363,Endeavor Air Inc.,52387940.0
3,1,9,3,9E,N840AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,1600-1659,83.0,1,6.0,70,2005.0,50.0,20363,Endeavor Air Inc.,52387940.0
4,1,10,4,9E,N8969A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,1600-1659,83.0,1,6.0,70,2004.0,50.0,20363,Endeavor Air Inc.,52387940.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567254,1,30,3,UA,N819UA,2024,14683,"San Antonio, TX",12266,"Houston, TX",...,1400-1459,191.0,1,3.0,7,1998.0,120.0,19977,United Air Lines Inc.,4823405.0
567255,1,30,3,UA,N37462,2022,14843,"San Juan, PR",12264,"Washington, DC",...,1400-1459,1571.0,7,2.0,9,2012.0,173.0,19977,United Air Lines Inc.,4372883.0
567256,1,30,3,UA,N37462,2021,12264,"Washington, DC",14843,"San Juan, PR",...,0800-0859,1571.0,7,1.0,36,2012.0,173.0,19977,United Air Lines Inc.,9363921.0
567257,1,30,3,UA,N26967,2020,12266,"Houston, TX",14771,"San Francisco, CA",...,1800-1859,1635.0,7,2.0,58,2016.0,219.0,19977,United Air Lines Inc.,20280380.0


In [39]:
data = pd.merge(data, monthly_airline_passengers, how='left', on='OP_UNIQUE_CARRIER')
data

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,...,DISTANCE,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,MANUFACTURE_YEAR,NUMBER_OF_SEATS,AIRLINE_ID,CARRIER_NAME,REV_PAX_ENP_110_x,REV_PAX_ENP_110_y
0,1,6,7,9E,N8694A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,83.0,1,3.0,76,2002.0,50.0,20363,Endeavor Air Inc.,52387940.0,14554161.0
1,1,7,1,9E,N8970D,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,83.0,1,6.0,71,2004.0,50.0,20363,Endeavor Air Inc.,52387940.0,14554161.0
2,1,8,2,9E,N820AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,83.0,1,5.0,69,2005.0,50.0,20363,Endeavor Air Inc.,52387940.0,14554161.0
3,1,9,3,9E,N840AY,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,83.0,1,6.0,70,2005.0,50.0,20363,Endeavor Air Inc.,52387940.0,14554161.0
4,1,10,4,9E,N8969A,3280,10397,"Atlanta, GA",11150,"Columbus, GA",...,83.0,1,6.0,70,2004.0,50.0,20363,Endeavor Air Inc.,52387940.0,14554161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567254,1,30,3,UA,N819UA,2024,14683,"San Antonio, TX",12266,"Houston, TX",...,191.0,1,3.0,7,1998.0,120.0,19977,United Air Lines Inc.,4823405.0,102019572.0
567255,1,30,3,UA,N37462,2022,14843,"San Juan, PR",12264,"Washington, DC",...,1571.0,7,2.0,9,2012.0,173.0,19977,United Air Lines Inc.,4372883.0,102019572.0
567256,1,30,3,UA,N37462,2021,12264,"Washington, DC",14843,"San Juan, PR",...,1571.0,7,1.0,36,2012.0,173.0,19977,United Air Lines Inc.,9363921.0,102019572.0
567257,1,30,3,UA,N26967,2020,12266,"Houston, TX",14771,"San Francisco, CA",...,1635.0,7,2.0,58,2016.0,219.0,19977,United Air Lines Inc.,20280380.0,102019572.0


### 2.5 Employee Data - `P10_EMPLOYEES` <a id='2.5'/>

This dataset contains the distibution of employees based on the job scope for each airline company. Some of the key column variables we will be using are the number of employees handling passengers (`PASSENGER_HANDLING`) and aircraft traffic handling (`GEN_ARCFT_TRAF_HANDLING`).

In [40]:
employees = pd.read_csv('raw_data/P10_EMPLOYEES.csv')
employees

,YEAR,AIRLINE_ID,OP_UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,CARRIER,CARRIER_NAME,ENTITY,GENERAL_MANAGE,PILOTS_COPILOTS,OTHER_FLT_PERS,...,GEN_ARCFT_TRAF_HANDLING,AIRCRAFT_CONTROL,PASSENGER_HANDLING,CARGO_HANDLING,TRAINEES_INTRUCTOR,STATISTICAL,TRAFFIC_SOLICITERS,OTHER,TRANSPORT_RELATED,TOTAL
0,2019,21352,0WQ,Avjet Corporation,0WQ,Avjet Corporation,D,4,53,6,...,0,0,0,3,1,18,0,7,0,161
1,2019,21492,1BQ,"Eastern Airlines f/k/a Dynamic Airways, LLC",1BQ,"Eastern Airlines f/k/a Dynamic Airways, LLC",I,14,50,0,...,0,0,0,0,1,13,0,3,0,161
2,2019,21712,2HQ,Elite Airways LLC,2HQ,Elite Airways LLC,D,9,32,0,...,0,0,0,0,0,7,0,0,0,123
3,2019,21974,3EQ,"Scott Aviation, LLC d/b/a Silver Air",3EQ,"Scott Aviation, LLC d/b/a Silver Air",D,0,29,0,...,0,0,0,0,0,0,0,0,0,69
4,2019,20408,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,D,14,54,11,...,0,0,0,0,4,45,5,20,0,347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2019,20207,XP,TEM Enterprises dba Avelo Airlines,XP,XTRA Airways,D,10,3,4,...,0,0,0,0,0,0,0,0,0,21
85,2019,20378,YV,Mesa Airlines Inc.,YV,Mesa Airlines Inc.,D,93,1312,0,...,55,57,0,0,4,93,0,109,0,2962
86,2019,20378,YV,Mesa Airlines Inc.,YV,Mesa Airlines Inc.,L,8,110,0,...,5,5,0,0,0,8,0,9,0,249
87,2019,20452,YX,Republic Airline,YX,Republic Airline,D,37,2444,19,...,0,183,23,0,67,267,0,260,0,6360


### Checking for NULL values

In [41]:
employees.isnull().sum()

YEAR                        0
AIRLINE_ID                  0
OP_UNIQUE_CARRIER           0
UNIQUE_CARRIER_NAME         0
CARRIER                     0
CARRIER_NAME                0
ENTITY                      0
GENERAL_MANAGE              0
PILOTS_COPILOTS             0
OTHER_FLT_PERS              0
PASS_GEN_SVC_ADMIN          0
MAINTENANCE                 0
ARCFT_TRAF_HANDLING_GRP1    0
GEN_ARCFT_TRAF_HANDLING     0
AIRCRAFT_CONTROL            0
PASSENGER_HANDLING          0
CARGO_HANDLING              0
TRAINEES_INTRUCTOR          0
STATISTICAL                 0
TRAFFIC_SOLICITERS          0
OTHER                       0
TRANSPORT_RELATED           0
TOTAL                       0
dtype: int64

### Check for duplicates in respective columns

In [42]:
# check for duplicates in key variables
print(employees['AIRLINE_ID'].nunique())
print(employees['OP_UNIQUE_CARRIER'].nunique())

49
49


In [43]:
employees = employees[['OP_UNIQUE_CARRIER','PASS_GEN_SVC_ADMIN','PASSENGER_HANDLING']]
employees = employees.groupby('OP_UNIQUE_CARRIER').sum().reset_index()
employees

,OP_UNIQUE_CARRIER,PASS_GEN_SVC_ADMIN,PASSENGER_HANDLING
0,0WQ,19,0
1,1BQ,41,0
2,2HQ,24,0
3,3EQ,32,0
4,5V,0,0
5,5X,0,0
6,5Y,273,0
7,8C,37,0
8,9E,1361,0
9,9S,3,0


## 2.6 Coordinates Data - `AIRPORT_COORDINATES` <a id='2.6'/>

This dataset contains the details of each airport such as the name, longitude and latitude.

In [44]:
coords = pd.read_csv('raw_data/AIRPORT_COORDINATES.csv')
coords

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,LATITUDE,LONGITUDE
0,10001,Afognak Lake Airport,58.109444,-152.906667
1,10003,Bear Creek Mining Strip,65.548056,-161.071667
2,10004,Lik Mining Camp,68.083333,-163.166667
3,10005,Little Squaw Airport,67.570000,-148.183889
4,10006,Kizhuyak Bay,57.745278,-152.882778
...,...,...,...,...
18128,16908,Deer Park Airport,47.966944,-117.428611
18129,16909,South Texas International at Edinburg,26.441667,-98.122222
18130,16910,Louisa County Freeman Field,38.009722,-77.970000
18131,16911,Caldwell Industrial,43.641944,-116.635833


### Check for null values

In [45]:
coords.isnull().sum()

ORIGIN_AIRPORT_ID       0
DISPLAY_AIRPORT_NAME    0
LATITUDE                1
LONGITUDE               1
dtype: int64

### Check for duplicate `ORIGIN_AIRPORT_ID`

In [46]:
coords[coords['ORIGIN_AIRPORT_ID'].duplicated(keep=False)].sort_values(by='ORIGIN_AIRPORT_ID')

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,LATITUDE,LONGITUDE
8,10010,Columbia County,42.288889,-73.710278
9,10010,Columbia County,42.291389,-73.710278
10,10011,Grand Canyon West,35.990278,-113.816389
11,10011,Grand Canyon West,35.986111,-113.816944
12,10011,Grand Canyon West,35.990278,-113.816389
...,...,...,...,...
18072,16854,Beja Airport,38.078889,-7.931111
18092,16874,Beijing Daxing International,39.496389,116.408611
18093,16874,Beijing Daxing International,39.499722,116.417500
18104,16885,Macklin,52.342778,-109.918889


### Drop duplicate `ORIGIN_AIRPORT_ID`

In [47]:
coords.drop_duplicates(subset='ORIGIN_AIRPORT_ID', inplace=True)
coords

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,LATITUDE,LONGITUDE
0,10001,Afognak Lake Airport,58.109444,-152.906667
1,10003,Bear Creek Mining Strip,65.548056,-161.071667
2,10004,Lik Mining Camp,68.083333,-163.166667
3,10005,Little Squaw Airport,67.570000,-148.183889
4,10006,Kizhuyak Bay,57.745278,-152.882778
...,...,...,...,...
18128,16908,Deer Park Airport,47.966944,-117.428611
18129,16909,South Texas International at Edinburg,26.441667,-98.122222
18130,16910,Louisa County Freeman Field,38.009722,-77.970000
18131,16911,Caldwell Industrial,43.641944,-116.635833


## 2.7 Weather Data - `airport_weather` <a id = '2.7' />

This dataset contains the weather statistics for each named location.

In [48]:
# import weather dataset
weather = pd.read_csv("raw_data/airport_weather_2019.csv")
weather.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,...,WT08,WT09,WESD,WT10,PSUN,TSUN,SN32,SX32,TOBS,WT11
0,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/1/2019,4.70,NaN,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/2/2019,4.92,NaN,0.57,0.0,0.0,56.0,59.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/3/2019,5.37,NaN,0.15,0.0,0.0,52.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/4/2019,12.08,NaN,1.44,0.0,0.0,56.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/5/2019,13.42,NaN,0.00,0.0,0.0,49.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
weather.shape

(38675, 32)

In [50]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38675 entries, 0 to 38674
Data columns (total 32 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  38675 non-null  object 
 1   NAME     38675 non-null  object 
 2   DATE     38675 non-null  object 
 3   AWND     38294 non-null  float64
 4   PGTM     3938 non-null   float64
 5   PRCP     38649 non-null  float64
 6   SNOW     26907 non-null  float64
 7   SNWD     26385 non-null  float64
 8   TAVG     28742 non-null  float64
 9   TMAX     38655 non-null  float64
 10  TMIN     38654 non-null  float64
 11  WDF2     38305 non-null  float64
 12  WDF5     38194 non-null  float64
 13  WSF2     38305 non-null  float64
 14  WSF5     38194 non-null  float64
 15  WT01     13851 non-null  float64
 16  WT02     1810 non-null   float64
 17  WT03     4783 non-null   float64
 18  WT04     266 non-null    float64
 19  WT05     119 non-null    float64
 20  WT06     406 non-null    float64
 21  WT07     25 

### Check for NULL values

In [51]:
# check for NULL values in weather data
weather.isnull().sum()

STATION        0
NAME           0
DATE           0
AWND         381
PGTM       34737
PRCP          26
SNOW       11768
SNWD       12290
TAVG        9933
TMAX          20
TMIN          21
WDF2         370
WDF5         481
WSF2         370
WSF5         481
WT01       24824
WT02       36865
WT03       33892
WT04       38409
WT05       38556
WT06       38269
WT07       38650
WT08       33950
WT09       38405
WESD       38669
WT10       38670
PSUN       38336
TSUN       38337
SN32       38310
SX32       38310
TOBS       38320
WT11       38674
dtype: int64

Based on the results above, there are 32 variables in the weather dataset. Most of the variables in the 'Weather' dataset have many NULL values. It is also known that not all the variables may be useful for our analysis. Therefore, we will be identifying those values which can be a useful, while dropping the rest.

`airport_weather_xxxx`

See _GHCND_documentation.pdf_ for full list
	
>   Important features:  
>	`NAME`: 			Location of reading  
>	`PRCP`: 			Inches of precipitation for day  
>	`SNOW`: 			Inches of snowfall for day  
>	`SNWD`: 			Inches of snow on ground for day  
>	`TMAX`: 			Max temperature for day  
>	`AWND`: 			Max wind speed for day  

### Dropping of unimportant features

In [52]:
weather.drop(['STATION','PGTM','TAVG','WDF2','WDF5','WSF2','WSF5','WT01','WT02','WT03','WT04','WT05','WT06','WT07','WT08','WT09','WESD','WT10','PSUN','TSUN','SN32','SX32','TOBS','WT11'],axis=1,inplace=True)
weather

,NAME,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN
0,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/1/2019,4.70,0.14,0.0,0.0,66.0,57.0
1,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/2/2019,4.92,0.57,0.0,0.0,59.0,49.0
2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/3/2019,5.37,0.15,0.0,0.0,55.0,51.0
3,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/4/2019,12.08,1.44,0.0,0.0,66.0,45.0
4,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/5/2019,13.42,0.00,0.0,0.0,59.0,44.0
...,...,...,...,...,...,...,...,...
38670,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-27,6.04,0.00,NaN,NaN,80.0,58.0
38671,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-28,5.37,0.06,NaN,NaN,74.0,64.0
38672,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-29,7.61,0.10,NaN,NaN,74.0,68.0
38673,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-30,5.82,0.02,NaN,NaN,72.0,47.0


In [53]:
weather.isnull().sum()

NAME        0
DATE        0
AWND      381
PRCP       26
SNOW    11768
SNWD    12290
TMAX       20
TMIN       21
dtype: int64

## 2.8 Airport List Data - `airports_list` <a id='2.8'/>

This dataset contains details of airports such as the name and city of origin.

In [54]:
airportList = pd.read_csv('raw_data/airports_list.csv')
airportList

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US"
1,10257,Albany International,"Albany, NY","ALBANY INTERNATIONAL AIRPORT, NY US"
2,10140,Albuquerque International Sunport,"Albuquerque, NM","ALBUQUERQUE INTERNATIONAL AIRPORT, NM US"
3,10299,Anchorage International,"Anchorage, AK","ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT, A..."
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
...,...,...,...,...
92,15370,Tulsa International,"Tulsa, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
93,12264,Washington Dulles International,"Washington, DC","WASHINGTON DULLES INTERNATIONAL AIRPORT, VA US"
94,13851,Will Rogers World,"Oklahoma City, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
95,12191,William P Hobby,"Houston, TX","HOUSTON WILLIAM P HOBBY AIRPORT, TX US"


In [55]:
airportList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ORIGIN_AIRPORT_ID     97 non-null     int64 
 1   DISPLAY_AIRPORT_NAME  97 non-null     object
 2   ORIGIN_CITY_NAME      97 non-null     object
 3   NAME                  97 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


### Check for NULL values

In [56]:
# check for null values
airportList.isnull().sum()

ORIGIN_AIRPORT_ID       0
DISPLAY_AIRPORT_NAME    0
ORIGIN_CITY_NAME        0
NAME                    0
dtype: int64

### Check for duplicate data

In [57]:
# check for duplicates
airportList.nunique()

ORIGIN_AIRPORT_ID       97
DISPLAY_AIRPORT_NAME    97
ORIGIN_CITY_NAME        94
NAME                    86
dtype: int64

In [58]:
# find the duplicated values in ORIGIN_CITY_NAME
airportList[airportList['ORIGIN_CITY_NAME'].duplicated(keep=False)]

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
9,13232,Chicago Midway International,"Chicago, IL","CHICAGO OHARE INTERNATIONAL AIRPORT, IL US"
10,13930,Chicago O'Hare International,"Chicago, IL","CHICAGO OHARE INTERNATIONAL AIRPORT, IL US"
32,12478,John F. Kennedy International,"New York, NY","LAGUARDIA AIRPORT, NY US"
37,12953,LaGuardia,"New York, NY","LAGUARDIA AIRPORT, NY US"
73,11278,Ronald Reagan Washington National,"Washington, DC","WASHINGTON DULLES INTERNATIONAL AIRPORT, VA US"
93,12264,Washington Dulles International,"Washington, DC","WASHINGTON DULLES INTERNATIONAL AIRPORT, VA US"


In [59]:
# find the duplicated values in NAME
airportList[airportList['NAME'].duplicated(keep=False)].sort_values(by='NAME')

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
9,13232,Chicago Midway International,"Chicago, IL","CHICAGO OHARE INTERNATIONAL AIRPORT, IL US"
10,13930,Chicago O'Hare International,"Chicago, IL","CHICAGO OHARE INTERNATIONAL AIRPORT, IL US"
28,12266,Honolulu International,"Honolulu, HI","HONOLULU INTERNATIONAL AIRPORT, HI US"
33,13830,Kahului Airport,"Kahului, HI","HONOLULU INTERNATIONAL AIRPORT, HI US"
32,12478,John F. Kennedy International,"New York, NY","LAGUARDIA AIRPORT, NY US"
37,12953,LaGuardia,"New York, NY","LAGUARDIA AIRPORT, NY US"
27,12173,Hollywood-Burbank Midpoint,"Burbank, CA","LOS ANGELES INTERNATIONAL AIRPORT, CA US"
41,12954,Long Beach Daugherty Field,"Long Beach, CA","LOS ANGELES INTERNATIONAL AIRPORT, CA US"
42,12892,Los Angeles International,"Los Angeles, CA","LOS ANGELES INTERNATIONAL AIRPORT, CA US"
55,13891,Ontario International,"Ontario, CA","LOS ANGELES INTERNATIONAL AIRPORT, CA US"


### Drop duplicate airport names

In [60]:
# drop duplicates
airportList = airportList.drop_duplicates(subset='NAME', keep='first')
airportList

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US"
1,10257,Albany International,"Albany, NY","ALBANY INTERNATIONAL AIRPORT, NY US"
2,10140,Albuquerque International Sunport,"Albuquerque, NM","ALBUQUERQUE INTERNATIONAL AIRPORT, NM US"
3,10299,Anchorage International,"Anchorage, AK","ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT, A..."
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
...,...,...,...,...
89,14307,Theodore Francis Green State,"Providence, RI","NEWPORT STATE AIRPORT, RI US"
90,13485,Truax Field,"Madison, WI","MADISON DANE CO REGIONAL AIRPORT, WI US"
92,15370,Tulsa International,"Tulsa, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
95,12191,William P Hobby,"Houston, TX","HOUSTON WILLIAM P HOBBY AIRPORT, TX US"


In [61]:
airportList.nunique()

ORIGIN_AIRPORT_ID       86
DISPLAY_AIRPORT_NAME    86
ORIGIN_CITY_NAME        86
NAME                    86
dtype: int64

## 3. Merge the Weather & Airport List Datasets <a id='3'/>

- Now that, we have cleaned the individual datasets, we can combine the datasets based on the similar column variable.
- Common variable = `NAME`

In [62]:
# Connect the two datasets
weather_airport = pd.merge(weather, airportList, how = 'left', on = 'NAME')
weather_airport

,NAME,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME
0,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/1/2019,4.70,0.14,0.0,0.0,66.0,57.0,10397.0,Atlanta Municipal,"Atlanta, GA"
1,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/2/2019,4.92,0.57,0.0,0.0,59.0,49.0,10397.0,Atlanta Municipal,"Atlanta, GA"
2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/3/2019,5.37,0.15,0.0,0.0,55.0,51.0,10397.0,Atlanta Municipal,"Atlanta, GA"
3,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/4/2019,12.08,1.44,0.0,0.0,66.0,45.0,10397.0,Atlanta Municipal,"Atlanta, GA"
4,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/5/2019,13.42,0.00,0.0,0.0,59.0,44.0,10397.0,Atlanta Municipal,"Atlanta, GA"
...,...,...,...,...,...,...,...,...,...,...,...
38670,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-27,6.04,0.00,NaN,NaN,80.0,58.0,14193.0,Pensacola Regional,"Pensacola, FL"
38671,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-28,5.37,0.06,NaN,NaN,74.0,64.0,14193.0,Pensacola Regional,"Pensacola, FL"
38672,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-29,7.61,0.10,NaN,NaN,74.0,68.0,14193.0,Pensacola Regional,"Pensacola, FL"
38673,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-30,5.82,0.02,NaN,NaN,72.0,47.0,14193.0,Pensacola Regional,"Pensacola, FL"


### Drop unnecessary columns

In [63]:
# Drop columns that are not useful other than 'ORIGIN_AIRPORT_ID' to connect the dataset
weather_airport.drop(['NAME','ORIGIN_CITY_NAME', 'DISPLAY_AIRPORT_NAME'], axis = 1, inplace = True)
weather_airport

,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,ORIGIN_AIRPORT_ID
0,1/1/2019,4.70,0.14,0.0,0.0,66.0,57.0,10397.0
1,1/2/2019,4.92,0.57,0.0,0.0,59.0,49.0,10397.0
2,1/3/2019,5.37,0.15,0.0,0.0,55.0,51.0,10397.0
3,1/4/2019,12.08,1.44,0.0,0.0,66.0,45.0,10397.0
4,1/5/2019,13.42,0.00,0.0,0.0,59.0,44.0,10397.0
...,...,...,...,...,...,...,...,...
38670,2019-12-27,6.04,0.00,NaN,NaN,80.0,58.0,14193.0
38671,2019-12-28,5.37,0.06,NaN,NaN,74.0,64.0,14193.0
38672,2019-12-29,7.61,0.10,NaN,NaN,74.0,68.0,14193.0
38673,2019-12-30,5.82,0.02,NaN,NaN,72.0,47.0,14193.0


In [64]:
# drop the rows with no ORIGIN_AIRPORT_ID
weather_airport.drop(weather_airport.loc[weather_airport['ORIGIN_AIRPORT_ID'].isna()].index, axis=0, inplace=True)

### Fill empty cells with mean value

In [65]:
# fill mean values for TMAX and AWND
weather_airport['TMAX'].fillna(round(weather_airport.groupby('ORIGIN_AIRPORT_ID')['TMAX'].transform('mean'), 1), inplace=True)
weather_airport['AWND'].fillna(round(weather_airport.groupby('ORIGIN_AIRPORT_ID')['AWND'].transform('mean'), 1), inplace=True)
weather_airport.fillna(0, inplace=True)

In [66]:
weather_airport.isnull().sum()

DATE                 0
AWND                 0
PRCP                 0
SNOW                 0
SNWD                 0
TMAX                 0
TMIN                 0
ORIGIN_AIRPORT_ID    0
dtype: int64

In [67]:
# add 'DAY_OF_MONTH' and 'MONTH' variables to match with other dataset
weather_airport['MONTH'] = pd.DatetimeIndex(weather_airport['DATE']).month
weather_airport['DAY_OF_MONTH'] = pd.DatetimeIndex(weather_airport['DATE']).day
weather_airport

,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,ORIGIN_AIRPORT_ID,MONTH,DAY_OF_MONTH
0,1/1/2019,4.70,0.14,0.0,0.0,66.0,57.0,10397.0,1,1
1,1/2/2019,4.92,0.57,0.0,0.0,59.0,49.0,10397.0,1,2
2,1/3/2019,5.37,0.15,0.0,0.0,55.0,51.0,10397.0,1,3
3,1/4/2019,12.08,1.44,0.0,0.0,66.0,45.0,10397.0,1,4
4,1/5/2019,13.42,0.00,0.0,0.0,59.0,44.0,10397.0,1,5
...,...,...,...,...,...,...,...,...,...,...
38670,2019-12-27,6.04,0.00,0.0,0.0,80.0,58.0,14193.0,12,27
38671,2019-12-28,5.37,0.06,0.0,0.0,74.0,64.0,14193.0,12,28
38672,2019-12-29,7.61,0.10,0.0,0.0,74.0,68.0,14193.0,12,29
38673,2019-12-30,5.82,0.02,0.0,0.0,72.0,47.0,14193.0,12,30


# 4. Merging of datasets <a id='4'/>

Let's create a function that cleans the monthly flight datasets to be merged with the other datasets above based on the similar column variable.

In [68]:
# create a function to clean the dataset for each month - ONTIME_REPORTING_##.csv
def clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport):
    
    # replace blanks with NaN
    df = df.replace(r'^\s*$', np.nan, regex=True)     
    
    # drop any unnamed columns (e.g. column 32)
    df.drop(df.columns[df.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)
    
    # drop columns
    df.drop(columns = ['CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY',
                         'SECURITY_DELAY','LATE_AIRCRAFT_DELAY'], axis=1, inplace=True)
    
    # drop cancelled flights as they are redundent for delay prediction
    df.drop(df.loc[df['CANCELLED']==1].index, axis=0, inplace=True)
    
    # drop rows with no TAIL_NUM
    df.drop(df.loc[df['TAIL_NUM'].isna()].index, axis=0, inplace=True)
    
    # Feature Engineering - manipulating data to create new variables
    # 'SEGMENT_NUMBER': the segment that this tail number is on for the day
    df["SEGMENT_NUMBER"] = df.groupby(["TAIL_NUM", 'DAY_OF_MONTH'])["DEP_TIME"].rank("dense", ascending=True)

    # 'CONCURRENT_FLIGHTS': number of concurrent flights leaving from the same airport and departure time block
    df['CONCURRENT_FLIGHTS'] = df.groupby(['ORIGIN_AIRPORT_ID','DAY_OF_MONTH', 'DEP_TIME_BLK'])['OP_UNIQUE_CARRIER'].transform("count")
    
    # ====== MERGE with aircraft ======
    df = pd.merge(df, aircraft, how='left', on='TAIL_NUM')
    
    # fill rows with empty number of seats with mean value
    # convert to int datatype
    df['NUMBER_OF_SEATS'].fillna((df['NUMBER_OF_SEATS'].mean()), inplace=True)
    df['NUMBER_OF_SEATS'] = df['NUMBER_OF_SEATS'].astype('int16')
    
    # ====== MERGE with airline ======
    df = pd.merge(df, airline, how='left', on='OP_UNIQUE_CARRIER')
    
    # Feature Engineering
    # AIRPORT_FLIGHTS_MONTH - average airport flights per month
    df['AIRPORT_FLIGHTS_MONTH'] = df.groupby(['ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')
    
    # AIRLINE_FLIGHTS_MONTH - average airline flights per month
    df['AIRLINE_FLIGHTS_MONTH'] = df.groupby(['OP_UNIQUE_CARRIER'])['ORIGIN_CITY_NAME'].transform('count')
    
    # AIRLINE_AIRPORT_FLIGHTS_MONTH - average flights per month for airline & airport
    df['AIRLINE_AIRPORT_FLIGHTS_MONTH'] = df.groupby(['OP_UNIQUE_CARRIER','ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')
    
    # ====== MERGE with monthly_airport_passengers & monthly_airline_passengers ======
    df = pd.merge(df, monthly_airport_passengers, how='left', on=['ORIGIN_AIRPORT_ID'])
    df = pd.merge(df, monthly_airline_passengers, how='left', on=['OP_UNIQUE_CARRIER'])

    # Feature Engineering
    # AVG_MONTHLY_PASS_AIRPORT - average passengers for airport for the month
    df['AVG_MONTHLY_PASS_AIRPORT'] = (df['REV_PAX_ENP_110_x']/12).astype('int64')
    
    # AVG_MONTHLY_PASS_AIRLINE - average passengers for airline for the month
    df['AVG_MONTHLY_PASS_AIRLINE'] = (df['REV_PAX_ENP_110_y']/12).astype('int64')
    
    # ====== MERGE with employee ======
    df = pd.merge(df, employees, how='left', on='OP_UNIQUE_CARRIER')
    
    # Feature Engineering
    # 'FLT_ATTENDANTS_PER_PASS' - number of flight attendants per passenger per airline
    df['FLT_ATTENDANTS_PER_PASS'] = df['PASSENGER_HANDLING']/df['REV_PAX_ENP_110_y']
    
    # 'GROUND_SERV_PER_PASS' - number of ground service employees per passenger per airline
    df['GROUND_SERV_PER_PASS'] = df['PASS_GEN_SVC_ADMIN']/df['REV_PAX_ENP_110_y']
    
    # 'PLANE_AGE' - the age of the plane
    df['MANUFACTURE_YEAR'].fillna((df['MANUFACTURE_YEAR'].mean()), inplace=True)
    df['PLANE_AGE'] = 2019 - df['MANUFACTURE_YEAR']
    
    # ====== MERGE with coords ======
    df = pd.merge(df, coords, how='left', on='ORIGIN_AIRPORT_ID')
    df['LATITUDE'] = round(df['LATITUDE'], 3)
    df['LONGITUDE'] = round(df['LONGITUDE'], 3)
    
    # Feature Engineering
    # 'PREVIOUS_AIRPORT'
    temp = df[['DAY_OF_MONTH','TAIL_NUM','DISPLAY_AIRPORT_NAME','SEGMENT_NUMBER']]
    df = pd.merge_asof(df.sort_values('SEGMENT_NUMBER'), temp.sort_values('SEGMENT_NUMBER'), 
                       on='SEGMENT_NUMBER', by=['DAY_OF_MONTH', 'TAIL_NUM'], 
                       allow_exact_matches=False)
    df['DISPLAY_AIRPORT_NAME_y'].fillna('NONE', inplace=True)
    df.rename(columns={'DISPLAY_AIRPORT_NAME_y':'PREVIOUS_AIRPORT', 
                       'DISPLAY_AIRPORT_NAME_x':'DEPARTING_AIRPORT'}, inplace=True)
    
    # ====== MERGE with weather_airport ======
    df = pd.merge(df, weather_airport, how='inner', on=['ORIGIN_AIRPORT_ID',
                                                        'MONTH','DAY_OF_MONTH'])
    
    # Drop columns not needed
    df.drop(columns = ['OP_UNIQUE_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID', 
                       'ORIGIN', 'ORIGIN_CITY_NAME', 'DEST_AIRPORT_ID', 'DEST', 'DEST_CITY_NAME', 
                       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME', 
                       'ARR_DELAY_NEW', 'ARR_TIME_BLK', 'CANCELLED', 'CANCELLATION_CODE', 'CRS_ELAPSED_TIME', 
                       'ACTUAL_ELAPSED_TIME', 'DISTANCE', 'MANUFACTURE_YEAR', 'AIRLINE_ID', 'REV_PAX_ENP_110_x', 
                       'REV_PAX_ENP_110_y', 'PASS_GEN_SVC_ADMIN', 'PASSENGER_HANDLING', 'DATE', 
                       'DAY_OF_MONTH'], axis = 1, inplace=True)
    
    # change data types to reduce memory usage
    df['MONTH'] = df['MONTH'].astype('object')
    df['DAY_OF_WEEK'] = df['DAY_OF_WEEK'].astype('object')
    df['DEP_DEL15'] = df['DEP_DEL15'].astype('int8')
    df['DISTANCE_GROUP'] = df['DISTANCE_GROUP'].astype('int8')
    df['SEGMENT_NUMBER'] = df['SEGMENT_NUMBER'].astype('int8')
    df['PLANE_AGE'] = df['PLANE_AGE'].astype('int32')
    
    # reset index
    df.reset_index(inplace=True, drop=True)
    
    return df

In [69]:
# apply the cleaning function to clean each month of flight data
# CAUTION: LONG RUN-TIME

# January 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_01.csv')
month1 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

# February 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_02.csv')
month2 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

# March 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_03.csv')
month3 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

# April 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_04.csv')
month4 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

# May 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_05.csv')
month5 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

# June 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_06.csv')
month6 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

# July 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_07.csv')
month7 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

# August 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_08.csv')
month8 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

# September 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_09.csv')
month9 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

# October 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_10.csv')
month10 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

# November 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_11.csv')
month11 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

# December 2019
df = pd.read_csv('raw_data/ONTIME_REPORTING_12.csv')
month12 = clean_month(df, aircraft, airline, monthly_airport_passengers, monthly_airline_passengers, 
                employees, coords, weather_airport)

In [70]:
# combine the cleaned datasets
cleaned_data = pd.concat([month1, month2, month3, month4, month5, month6, month7, month8, month9, month10, 
                          month11, month12]).reset_index(drop=True)

In [71]:
cleaned_data.shape

(5625398, 27)

In [72]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5625398 entries, 0 to 5625397
Data columns (total 27 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   MONTH                          object 
 1   DAY_OF_WEEK                    object 
 2   DEP_DEL15                      int8   
 3   DEP_TIME_BLK                   object 
 4   DISTANCE_GROUP                 int8   
 5   SEGMENT_NUMBER                 int8   
 6   CONCURRENT_FLIGHTS             int64  
 7   NUMBER_OF_SEATS                int16  
 8   CARRIER_NAME                   object 
 9   AIRPORT_FLIGHTS_MONTH          int64  
 10  AIRLINE_FLIGHTS_MONTH          int64  
 11  AIRLINE_AIRPORT_FLIGHTS_MONTH  int64  
 12  AVG_MONTHLY_PASS_AIRPORT       int64  
 13  AVG_MONTHLY_PASS_AIRLINE       int64  
 14  FLT_ATTENDANTS_PER_PASS        float64
 15  GROUND_SERV_PER_PASS           float64
 16  PLANE_AGE                      int32  
 17  DEPARTING_AIRPORT              object 
 18  LA

### Check for NULL variables one last time

In [73]:
# check for any null values in cleaned dataset
cleaned_data.isnull().sum()

MONTH                            0
DAY_OF_WEEK                      0
DEP_DEL15                        0
DEP_TIME_BLK                     0
DISTANCE_GROUP                   0
SEGMENT_NUMBER                   0
CONCURRENT_FLIGHTS               0
NUMBER_OF_SEATS                  0
CARRIER_NAME                     0
AIRPORT_FLIGHTS_MONTH            0
AIRLINE_FLIGHTS_MONTH            0
AIRLINE_AIRPORT_FLIGHTS_MONTH    0
AVG_MONTHLY_PASS_AIRPORT         0
AVG_MONTHLY_PASS_AIRLINE         0
FLT_ATTENDANTS_PER_PASS          0
GROUND_SERV_PER_PASS             0
PLANE_AGE                        0
DEPARTING_AIRPORT                0
LATITUDE                         0
LONGITUDE                        0
PREVIOUS_AIRPORT                 0
AWND                             0
PRCP                             0
SNOW                             0
SNWD                             0
TMAX                             0
TMIN                             0
dtype: int64

In [74]:
# export the cleaned dataset
cleaned_data.to_csv('cleaned_data.csv', index=False)

Documentation of cleaned dataset:
> **MONTH**:				Month  
> **DAY_OF_WEEK**:			Day of Week  
> **DEP_DEL15**: 			TARGET Binary of a departure delay over 15 minutes (1 is yes)  
> **DEP_TIME_BLOCK**:			Departure block  
> **DISTANCE_GROUP**:			Distance group to be flown by departing aircraft  
> **SEGMENT_NUMBER**:			The segment that this tail number is on for the day  
> **CONCURRENT_FLIGHTS**:		Concurrent flights leaving from the airport in the same departure block  
> **NUMBER_OF_SEATS**:		Number of seats on the aircraft  
> **CARRIER_NAME**:			Carrier  
> **AIRPORT_FLIGHTS_MONTH**:		Avg Airport Flights per Month  
> **AIRLINE_FLIGHTS_MONTH**:		Avg Airline Flights per Month  
> **AIRLINE_AIRPORT_FLIGHTS_MONTH**:	Avg Flights per month for Airline AND Airport  
> **AVG_MONTHLY_PASS_AIRPORT**:	Avg Passengers for the departing airport for the month  
> **AVG_MONTHLY_PASS_AIRLINE**:	Avg Passengers for airline for month  
> **FLT_ATTENDANTS_PER_PASS**:	Flight attendants per passenger for airline  
> **GROUND_SERV_PER_PASS**:		Ground service employees (service desk) per passenger for airline  
> **PLANE_AGE**:			Age of departing aircraft  
> **DEPARTING_AIRPORT**:		Departing Airport  
> **LATITUDE**:			Latitude of departing airport  
> **LONGITUDE**:			Longitude of departing airport  
> **PREVIOUS_AIRPORT**:		Previous airport that aircraft departed from  
> **AWND**:				Max wind speed for day  
> **PRCP**:				Inches of precipitation for day  
> **SNOW**:				Inches of snowfall for day  
> **SNWD**:				Inches of snow on ground for day  
> **TMAX**:				Max temperature for day  
> **TMIN**:             Min temperature for day

# 5. Data Cleansing Techniques used <a id='5'/>

1. Handling NaN values - dropping them or filling with mean values
2. Dropping irrelevant columns & duplicate rows
3. Feature Engineering
4. Changing data types (to reduce memory usage)
5. Encoding of categorical variables